In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import time, base64, json, requests, asyncio
import nest_asyncio
nest_asyncio.apply()
from typing import List, Dict, Union, Any 

# Import our providers
from llm_master import QueryLLM, LLMConfig
config = LLMConfig.from_env()
llm = QueryLLM(config)


/Users/sina/Desktop/Uflo Platform/uflo-AI-server/ai-server/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:llm_master.response_synthesizer:Initialized QueryLLM handler with rate limiters


In [ ]:


async def run_stream():
    messages = [
        {
            "role": "user", 
            "content": "Tell me a story about a cat",
        },
    ]

    try:
        response_generator = await llm.query(
            # model_name="o3-mini",
            # reasoning_effort="low",
            model_name="gemini-2.0-flash",
            # model_name="accounts/fireworks/models/deepseek-r1",
            messages=messages,
            stream=True,
            fallback_provider="openai",
            fallback_model="gpt-4o",
            moderation=False
        )
        
        async for chunk in response_generator:
            print(chunk, end="", flush=True)
            
    except Exception as e:
        print(f"Error: {str(e)}")

# This line is key in Jupyter
await run_stream()  # Don't use asyncio.run() here

In [ ]:


async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            # model_name="accounts/fireworks/models/deepseek-r1",
            model_name="gemini-2.0-flash",
            # reasoning_effort="low",
            messages=messages,
            stream=False,  # Set to False for non-streaming
            # temperature=0.5,
            fallback_provider="openai",
            fallback_model="gpt-4o",
            moderation=False
        )
        
        # Print the full response content
        print(response.content)
        
        # You can also access other metadata
        print("\n--- Response Metadata ---")
        print(f"Model: {response.model_name}")
        print(f"Input tokens: {response.usage.input_tokens}")
        print(f"Output tokens: {response.usage.output_tokens}")
        print(f"Cost: ${response.cost:.6f}")
        print(f"Latency: {response.latency:.2f} seconds")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [ ]:

# Using messages format
response1 = await llm.query(
    model_name="recraftv3",
    messages=[{"role": "user", "content": "race car on a track"}],
    style="digital_illustration"
)


print(f"Image URL: {response1.content}")

In [ ]:
# Define the prompt
prompt = "A cat on its back legs running like a human is holding a big silver fish with its arms. The cat is running away from the shop owner and has a panicked look on his face. The scene is situated in a crowded market."

try:
    # Generate the image
    response = await llm.query(
        model_name="flux-pro-1.1",
        messages=[{"role": "user", "content": prompt}],
        width=1024,  # Optional: specify image width
        height=768,   # Optional: specify image height
        prompt_upsampling=True, #If active, automatically modifies the prompt for more creative generation.
        guidance=3 #High guidance scales improve prompt adherence at the cost of reduced realism.
    )
    
    # Print the result (which is the image URL)
    print(f"Image generation successful! URL: {response.content}")
    
except Exception as e:
    print(f"Error generating image: {str(e)}")

In [4]:

async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            model_name="gpt-4o-mini-audio-preview",
            messages=messages,
            stream=False,
            modality=["text", "audio"],
            audio={"voice": "ash", "format": "wav"}
        )
        
         # Print the text response
        print("Text response:")
        print(response.content)
        
        # Save the audio to a file if available
        if response.audio_data:
            wav_bytes = base64.b64decode(response.audio_data)
            output_file = "dog_response.wav"
            with open(output_file, "wb") as f:
                f.write(wav_bytes)
            print(f"\nAudio saved to '{output_file}'")
        else:
            print("\nNo audio data received in the response")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

Text response:
None

Audio saved to 'dog_response.wav'
